In [ ]:
import sys
sys.path.append('C:\\Users\\q524745\\Documents\\bachelor_thesis\\.venv\\lib\\site-packages')

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
import torch.nn.functional as F
import pytorch_lightning as pl

from transformers import AutoTokenizer, AutoModel, AdamW, get_cosine_schedule_with_warmup
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchmetrics.functional.classification import auroc

In [ ]:
train_data = pd.read_csv("train.csv")
train_path = "train.csv"

val_data = pd.read_csv("val.csv")
val_path = "val.csv"

# train_data.head(2)

In [ ]:
# creating a new column, switching around healthy  & unhealthy ( only for exploration purposes, can be removed!)
train_data['unhealthy'] = np.where(train_data['healthy'] == 1, 0, 1)

attributes = ['antagonize', 'condescending', 'dismissive', "generalisation", "generalisation_unfair", "hostile", "unhealthy"]

# train_data[attributes].sum().plot.bar()

## Pytorch dataset

In [ ]:
class UCC_Dataset(Dataset):
    """
    @param sample: Because it is a highly unbalanced dataset, using the sample size manually cuts of the majority class once the max is reached
    """
    def __init__(self, data_path, tokenizer, attributes, max_token_length: int = 512, sample = 5000):
        self.data_path = data_path
        self.tokenizer = tokenizer
        self.attributes = attributes
        self.max_token_length = max_token_length
        self.sample = sample
        self.__prepare_data()

    ## ( __ makes it a private funct)
    def __prepare_data(self):
        data = pd.read_csv(self.data_path)
        data['unhealthy'] = np.where(data['healthy'] == 1, 0, 1)

        if self.sample is not None:
            # if there is a positive attribute
            unhealthy = data.loc[data[attributes].sum(axis=1) > 0 ]
            healthy = data.loc[data[attributes].sum(axis=1) == 0 ]
            # random state for reproduction
            self.data = pd.concat([unhealthy, healthy.sample(self.sample, random_state=666)])

        else:
            self.data = data


    def __len__(self):
        return(len(self.data))


    def __getitem__(self, index):
        item = self.data.iloc[index]
        comment = str(item.comment)

        # turn things into a tensor
        attributes = torch.FloatTensor(item[self.attributes])
        # takes input as string, the "special tokens" ensures formatting by adding beggining & end of sentence tokens
        # Truncate: comments to max_length
        # padding: pad tokens < 512 to may_length
        # return_atention-mask: returns 1 everywhere except for paddings (0)
        tokens = self.tokenizer.encode_plus (comment,
                                                add_special_tokens = True,
                                                return_tensors="pt", 
                                                truncation =True,
                                                max_length = self.max_token_length,
                                                padding = "max_length",
                                                return_attention_mask=True)
        
        return {'input_ids': tokens.input_ids.flatten(), 'attention_mask': tokens.attention_mask.flatten(), 'labels': attributes }

In [ ]:
# test it (== create everything): 

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
ucc_ds = UCC_Dataset(train_path, tokenizer, attributes)

ucc_ds_val = UCC_Dataset(val_path, tokenizer, attributes, sample =None)

In [ ]:
# taking a look at the first item
len(ucc_ds)
# ucc_ds.__getitem__(0)

Cell Output: 


{'input_ids':

 tensor([    0, 19897, 29846,     4,    38,    64,    75,   679, 46301,    74, 9802,   132,     9,    39,   308,    95,     7,  1471,   559,   332, 8,   146,  7064,   356,  1099,     4,   370,   214,   235,     6,46301,    16,    10, 34759,     4,     2,     1,     1,     1,     1, 1,     1,     1,     1,     1,     1,     1,     1,     1,     1, 1,     1,     1,     1,     1,     1,     1,     1,     1,     1, 1,     1,     1,     1,     1,     1,     1,     1,     1,     1,1,     1,     1,     1,     1,     1,     1,     1,     1,     1,1,     1,     1,     1,     1,     1,     1,     1,     1,     1,1,     1,     1,     1,     1,     1,     1,     1,     1,     1,1,     1,     1,     1,     1,     1,     1,     1,     1,     1,1,     1,     1,     1,     1,     1,     1,     1,     1,     1,1,     1,     1,     1,     1,     1,     1,     1,     1,     1,1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,             1,     1]),
 'attention_mask': 
 
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': 
 
 tensor([0., 0., 0., 0., 0., 0., 1.])}


Explanantion:
- attention_mask:  1 = actual token: 0 = padding
- input_ids: 0 = Start-of_sentence-Token, 2 = End-of-sentence-token, 1 = Padding token

In [ ]:
len(ucc_ds)

## Data module

The LightningDataModule is a convenient way to manage data in PyTorch Lightning.
It encapsulates training, validation, testing, and prediction dataloaders, as well as any necessary steps for data processing, downloads, and transformations. By using a LightningDataModule, you can easily develop dataset-agnostic models, hot-swap different datasets, and share data splits and transformations across projects

In [ ]:
# technically this is a convenience thing, not necessary for functionality

class UCC_Data_Module(pl.LightningDataModule):

    def __init__(self, train_path, val_path, attributes, batch_size: int = 16, max_token_len: int =512, model_name = "roberta-base"):
        super().__init__()
        self.train_path = train_path
        self.val_path = val_path
        self.attributes = attributes
        self.batch_size = batch_size
        self.max_token_len = max_token_len
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def setup(self, stage = None):
        # "fit" = training stage
        if stage in (None, "fit"):
            self.train_dataset = UCC_Dataset(self.train_path, self.tokenizer, self.attributes)
            self.val_dataset = UCC_Dataset(self.val_path, self.tokenizer, self.attributes, sample = None)
        
        if stage == "predict":
            self.val_dataset = UCC_Dataset(self.val_path, self.tokenizer, self.attributes, sample = None)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size = self.batch_size, num_workers=2, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size = self.batch_size, num_workers=2, shuffle=False)
    
    def predict_dataloader(self):
        return DataLoader(self.val_dataset, batch_size = self.batch_size, num_workers=2, shuffle=False)

In [ ]:
# testing Loader

ucc_data_module = UCC_Data_Module(train_path, val_path, attributes)

In [ ]:
# create datasets
ucc_data_module.setup()

In [ ]:
dl = ucc_data_module.train_dataloader()

# returns length (= number of batches (a 16)) in the training data set
# samples in training data: 9299 / 16 = 581,18 (0,18 gets filled to only have full batches)
len(dl)

## Model

https://huggingface.co/docs/transformers/en/main_classes/optimizer_schedules

--> get_cosine_with_warmup
Start with a learning rate of 0, increase fast, then decrease slowly to hone in on optimal solution 
- auroc technically not good for unbalanced, but still better than accuracy for this data (cause 95% == healthy, so accuraccy of 95% could also mean everything gets predicted as Healthy)

Theory: 

https://www.youtube.com/watch?v=vNKIg8rXK6w
->  @ 48:00 for option explanation
-> VERY HELPFUL IMAGE!!!!

In [ ]:
class UCC_Classifier(pl.LightningModule):
    def __init__(self, config: dict):
        super().__init__()
        self.config = config
        self.pretrained_model = AutoModel.from_pretrained(config["model_name"], return_dict = True)
        
        # appending a classification layer (="head") & a hidden layer before final_layer
        self.hidden = nn.Linear(self.pretrained_model.config.hidden_size, self.pretrained_model.config.hidden_size )
        self.classifier = nn.Linear(self.pretrained_model.config.hidden_size, self.config["n_labels"])
        
        # initializing custom layers, optional, would also be done automatically, but better performance if stated explicitly)
        torch.nn.init.xavier_uniform_(self.hidden.weight)
        torch.nn.init.xavier_uniform_(self.classifier.weight)

        # loss function
        self.loss_func = nn.BCEWithLogitsLoss(reduction="mean")

        # dropout layer (ensures no overfitting); randomly turns on/off random nodes for each training loop; so  model is not dependant on specific node
        self.dropout = nn.Dropout()

    # Forward pass, labels only needed while training, not for prediction, so set to none by default
    def forward(self, input_ids, attention_mask, labels = None):
        # roberta model
        output = self.pretrained_model(input_ids = input_ids, attention_mask = attention_mask)

        # output is given for every single token (512 per batch for me), so: take mean from all
        # there is the option to usepooled Output, which uses the first and last few tokens, depends on usecase
        pooled_output = torch.mean(output.last_hidden_state, 1)

        # NN-layers
        pooled_output = self.hidden(pooled_output)  # pass sentence(or however tokens are pooled) through hidden layer
        pooled_output = self.dropout(pooled_output) # pass through dropout layer
        pooled_output = F.relu(pooled_output)   # pass through activatioin function( relu; in this case)
        logits = self.classifier(pooled_output)     # pass through classification layer

        # calculate loss
        loss = 0

        if labels is not None:
            loss = self.loss_func(logits.view(-1, self.config["n_labels"]), labels.view(-1, self.config["n_labels"]))
        
        # logits = fancy name for model output
        return loss, logits
    
    def training_step(self, batch, batch_index):
        # get & unpack the output dict ( as in.  ucc_ds.__getitem__(0) into self)
        loss, logits = self(**batch)
        # save training loss
        self.log("train_loss", loss, prog_bar=True, logger= True)
        return {"loss": loss, "predictions": logits, "labels": batch["labels"]}
    

    def validation_step(self, batch, batch_index):
        loss, logits = self(**batch)
        self.log("validation_loss", loss, prog_bar=True, logger= True)
        return {"val_loss": loss, "predictions": logits, "labels": batch["labels"]}
    

    def prediction_step(self, batch, batch_index):
        __, logits = self(**batch)
        # prediction:
        return logits

    # cofigure optimizers
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.config["lr"], weight_decay=self.config["w_decay"])
        # Scheduler: 
        total_steps = self.config["train_size"] / self.config["bs"]
        # warmup in config is a percentage
        warmup_steps = math.floor(total_steps * self.config["warmup"])

        scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
        return [optimizer], [scheduler]

In [ ]:
# Params can be meddled with, eg. using regular roberts, ...

config = {
    'model_name': "distilroberta-base",
    "n_labels": len(attributes), 
    "bs": 64,              # batch size 
    "lr": 1.5e-6,
    "warmup": 0.2,
    "train_size": len(ucc_data_module.train_dataloader()), 
    "w_decay": 0.001,
    "n_epochs": 1
    }

In [ ]:
model = UCC_Classifier(config)

In [ ]:
idx = 0
input_ids = ucc_ds.__getitem__(idx)["input_ids"]
# attention mask
am = ucc_ds.__getitem__(idx)["attention_mask"]
labels = ucc_ds.__getitem__(idx)["labels"]

# unsqueeze necessary because only one example here, expected are multiple
loss, output = model(input_ids.unsqueeze(dim=0), am.unsqueeze(dim=0), labels.unsqueeze(dim=0))

In [ ]:
print(loss, output)

# example output: 
# tensor(0.7984, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>) tensor([[-0.5230,  0.2949,  0.2186, -0.2637,  0.4882, -0.3237, -1.0935]],grad_fn=<AddmmBackward0>)

## Training (Putting it all together)

In [ ]:
import sys
sys.path.append('C:\\Users\\q524745\\Documents\\bachelor_thesis\\.venv\\lib\\site-packages')

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
import torch.nn.functional as F
import pytorch_lightning as pl
import multiprocessing
import warnings

from transformers import AutoTokenizer, AutoModel, AdamW, get_cosine_schedule_with_warmup
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchmetrics.functional.classification import auroc
from transformers import logging as transformers_logging


warnings.filterwarnings('ignore', category=FutureWarning)
transformers_logging.set_verbosity_error()
warnings.filterwarnings(
    'ignore',
    message="The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck."
)

train_data = pd.read_csv("train.csv")
train_path = "train.csv"

val_data = pd.read_csv("val.csv") 
val_path = "val.csv"

# train_data.head(2)

# creating a new column, switching around healthy  & unhealthy ( only for exploration purposes, can be removed!)
train_data['unhealthy'] = np.where(train_data['healthy'] == 1, 0, 1)

attributes = ['antagonize', 'condescending', 'dismissive', "generalisation", "generalisation_unfair", "hostile", "unhealthy"]

# train_data[attributes].sum().plot.bar()

class UCC_Dataset(Dataset):
    """
    @param sample: Because it is a highly unbalanced dataset, using the sample size manually cuts of the majority class once the max is reached
    """
    def __init__(self, data_path, tokenizer, attributes, max_token_length: int = 64, sample = 5000):
        self.data_path = data_path
        self.tokenizer = tokenizer
        self.attributes = attributes
        self.max_token_length = max_token_length
        self.sample = sample
        self.__prepare_data()

    ## ( __ makes it a private funct)
    def __prepare_data(self):
        data = pd.read_csv(self.data_path)
        data['unhealthy'] = np.where(data['healthy'] == 1, 0, 1)

        if self.sample is not None:
            # if there is a positive attribute
            unhealthy = data.loc[data[attributes].sum(axis=1) > 0 ]
            healthy = data.loc[data[attributes].sum(axis=1) == 0 ]
            # random state for reproduction
            self.data = pd.concat([unhealthy, healthy.sample(self.sample, random_state=666)])

        else:
            self.data = data


    def __len__(self):
        return(len(self.data))


    def __getitem__(self, index):
        item = self.data.iloc[index]
        comment = str(item.comment)

        # turn things into a tensor
        attributes = torch.FloatTensor(item[self.attributes])
        # takes input as string, the "special tokens" ensures formatting by adding beggining & end of sentence tokens
        # Truncate: comments to max_length
        # padding: pad tokens < 512 to may_length
        # return_atention-mask: returns 1 everywhere except for paddings (0)
        tokens = self.tokenizer.encode_plus (comment,
                                                add_special_tokens = True,
                                                return_tensors="pt", 
                                                truncation =True,
                                                max_length = self.max_token_length,
                                                padding = "max_length",
                                                return_attention_mask=True)
        
        return {'input_ids': tokens.input_ids.flatten(), 'attention_mask': tokens.attention_mask.flatten(), 'labels': attributes }
    

# test it (== create everything): 

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
ucc_ds = UCC_Dataset(train_path, tokenizer, attributes)

ucc_ds_val = UCC_Dataset(val_path, tokenizer, attributes, sample =None)

# technically this is a convenience thing, not necessary for functionality

class UCC_Data_Module(pl.LightningDataModule):

    def __init__(self, train_path, val_path, attributes, batch_size: int = 16, max_token_len: int =64, model_name = "roberta-base"):
        super().__init__()
        self.train_path = train_path
        self.val_path = val_path
        self.attributes = attributes
        self.batch_size = batch_size
        self.max_token_len = max_token_len
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def setup(self, stage = None):
        # "fit" = training stage
        if stage in (None, "fit"):
            self.train_dataset = UCC_Dataset(self.train_path, self.tokenizer, self.attributes)
            self.val_dataset = UCC_Dataset(self.val_path, self.tokenizer, self.attributes, sample = None)
        
        if stage == "predict":
            self.val_dataset = UCC_Dataset(self.val_path, self.tokenizer, self.attributes, sample = None)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size = self.batch_size, num_workers=0, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size = self.batch_size, num_workers=0, shuffle=False)
    
    def predict_dataloader(self):
        return DataLoader(self.val_dataset, batch_size = self.batch_size, num_workers=0, shuffle=False)
    
    # testing Loader

ucc_data_module = UCC_Data_Module(train_path, val_path, attributes)
# create datasets
ucc_data_module.setup()

class UCC_Classifier(pl.LightningModule):
    def __init__(self, config: dict):
        super().__init__()
        self.config = config
        self.pretrained_model = AutoModel.from_pretrained(config["model_name"], return_dict = True)
        
        # appending a classification layer (="head") & a hidden layer before final_layer
        self.hidden = nn.Linear(self.pretrained_model.config.hidden_size, self.pretrained_model.config.hidden_size )
        self.classifier = nn.Linear(self.pretrained_model.config.hidden_size, self.config["n_labels"])
        
        # initializing custom layers, optional, would also be done automatically, but better performance if stated explicitly)
        torch.nn.init.xavier_uniform_(self.hidden.weight)
        torch.nn.init.xavier_uniform_(self.classifier.weight)

        # loss function
        self.loss_func = nn.BCEWithLogitsLoss(reduction="mean")

        # dropout layer (ensures no overfitting); randomly turns on/off random nodes for each training loop; so  model is not dependant on specific node
        self.dropout = nn.Dropout()

    # Forward pass, labels only needed while training, not for prediction, so set to none by default
    def forward(self, input_ids, attention_mask, labels = None):
        # roberta model
        output = self.pretrained_model(input_ids = input_ids, attention_mask = attention_mask)

        # output is given for every single token (512 per batch for me), so: take mean from all
        # there is the option to usepooled Output, which uses the first and last few tokens, depends on usecase
        pooled_output = torch.mean(output.last_hidden_state, 1)

        # NN-layers
        pooled_output = self.hidden(pooled_output)  # pass sentence(or however tokens are pooled) through hidden layer
        pooled_output = self.dropout(pooled_output) # pass through dropout layer
        pooled_output = F.relu(pooled_output)   # pass through activatioin function( relu; in this case)
        logits = self.classifier(pooled_output)     # pass through classification layer

        # calculate loss
        loss = 0

        if labels is not None:
            loss = self.loss_func(logits.view(-1, self.config["n_labels"]), labels.view(-1, self.config["n_labels"]))
        
        # logits = fancy name for model output
        return loss, logits
    
    def training_step(self, batch, batch_index):
        # get & unpack the output dict ( as in.  ucc_ds.__getitem__(0) into self)
        loss, logits = self(**batch)
        # save training loss
        self.log("train_loss", loss, prog_bar=True, logger= True)
        return {"loss": loss, "predictions": logits, "labels": batch["labels"]}
    

    def validation_step(self, batch, batch_index):
        loss, logits = self(**batch)
        self.log("validation_loss", loss, prog_bar=True, logger= True)
        return {"val_loss": loss, "predictions": logits, "labels": batch["labels"]}
    

    def prediction_step(self, batch, batch_index):
        __, logits = self(**batch)
        # prediction:
        return logits

    # cofigure optimizers
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.config["lr"], weight_decay=self.config["w_decay"])
        # Scheduler: 
        total_steps = self.config["train_size"] / self.config["bs"]
        # warmup in config is a percentage
        warmup_steps = math.floor(total_steps * self.config["warmup"])

        scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
        return [optimizer], [scheduler]
    
# Params can be meddled with, eg. using regular roberta, ...

config = {
    'model_name': "distilroberta-base",
    "n_labels": len(attributes), 
    "bs": 64,              # batch size 
    "lr": 1.5e-6,
    "warmup": 0.2,
    "train_size": len(ucc_data_module.train_dataloader()), 
    "w_decay": 0.001,
    "n_epochs": 1
    }   

if __name__ == '__main__':

    multiprocessing.set_start_method('spawn', force=True)

    # create necessary dataset using datamodule
    ucc_data_module = UCC_Data_Module(train_path, val_path, attributes, batch_size=config["bs"])
    ucc_data_module.setup()

    # model 
    model = UCC_Classifier(config)

    # training
    trainer = pl.Trainer(max_epochs=config["n_epochs"], num_sanity_val_steps=10 )   # num_sanity_val_steps: check in beginning for correct setup
    trainer.fit(model, ucc_data_module)


## Predict / Evaluate

In [3]:
%load_ext tensorboard
%reload_ext tensorboard
%tensorboard dev upload --logdir ./lightning_logs/


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 17276.